In [1]:
!pip install -q --upgrade pip
!pip install -q transformers datasets peft accelerate bitsandbytes==0.43.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
import torch

# Load dataset from Hugging Face
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")

# Split into train and test
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Print basic info
print(train_dataset)
print(test_dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'is_correct', 'answer', 'solution'],
    num_rows: 1000000
})
Dataset({
    features: ['question', 'is_correct', 'answer', 'solution'],
    num_rows: 10000
})


In [3]:
# Reduce the dataset to avoid GPU overload
small_train = train_dataset.shuffle(seed=42).select(range(12000))
small_test = test_dataset

# Check one example and the sizes
print(small_train[0])
print(len(small_train), len(small_test))


{'question': 'A line is parameterized by\n\\[\\begin{pmatrix} x \\\\ y \\end{pmatrix} = \\begin{pmatrix} 2 \\\\ 3 \\end{pmatrix} + t \\begin{pmatrix} -1 \\\\ 5 \\end{pmatrix}.\\]A second line is parameterized by\n\\[\\begin{pmatrix} x \\\\ y \\end{pmatrix} = \\begin{pmatrix} 0 \\\\ 7 \\end{pmatrix} + u \\begin{pmatrix} -1 \\\\ 4 \\end{pmatrix}.\\]Find the point where the lines intersect.', 'is_correct': False, 'answer': '(2/3,4/3)', 'solution': "First, we need to solve the system of equations\n\\[\n\\begin{aligned}\n2 - t &= s\\\\\n3 + 5t &= 7 + 4s\n\\end{aligned}\n\\]\nby eliminating s.\nWe'll use sympy.\n<llm-code>\nfrom sympy import symbols, solve\n\n# define the variables\nt, s = symbols('t s')\n\n# define the equations\nequation1 = 2 - t - s\nequation2 = 3 + 5*t - (7 + 4*s)\n\n# solve the equations\nsolutions = solve([equation1, equation2], (t, s))\nprint(solutions)\n</llm-code>\n<llm-code-output>\n{s: 2/3, t: 4/3}\n</llm-code-output>\nTherefore the point where the lines intersect

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Choose model
model_name = "microsoft/deberta-v3-small"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

In [5]:
def preprocess_function(batch):
    texts = [
        "Question: " + q + " Answer: " + a
        for q, a in zip(batch["question"], batch["answer"])
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

# Tokenize
tokenized_train = small_train.map(preprocess_function, batched=True, batch_size=256)
tokenized_test = small_test.map(preprocess_function, batched=True, batch_size=256)

# Convert True/False to integers (0/1)
def encode_labels(example):
    example["is_correct"] = 1 if example["is_correct"] else 0
    return example

tokenized_train = tokenized_train.map(encode_labels)

# Prepare for PyTorch
tokenized_train = tokenized_train.rename_column("is_correct", "labels")
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    do_eval=False,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train
)


In [7]:
trainer.train()


Step,Training Loss
500,0.558600
1000,0.509800
1500,0.494600


TrainOutput(global_step=1500, training_loss=0.5210113525390625, metrics={'train_runtime': 260.7945, 'train_samples_per_second': 46.013, 'train_steps_per_second': 5.752, 'total_flos': 794832703488000.0, 'train_loss': 0.5210113525390625, 'epoch': 1.0})

In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# First, encode test labels
def encode_labels(example):
    example["is_correct"] = 1 if example["is_correct"] else 0
    return example

tokenized_test = tokenized_test.map(encode_labels)
tokenized_test = tokenized_test.rename_column("is_correct", "labels")
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Predict
preds = trainer.predict(tokenized_test)

# Convert logits → predicted labels
pred_labels = np.argmax(preds.predictions, axis=1)

# Compute metrics
print("✅ Accuracy:", accuracy_score(preds.label_ids, pred_labels))
print(classification_report(preds.label_ids, pred_labels))


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

✅ Accuracy: 0.4466
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.45      0.62     10000

    accuracy                           0.45     10000
   macro avg       0.50      0.22      0.31     10000
weighted avg       1.00      0.45      0.62     10000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
from collections import Counter
Counter(tokenized_test["labels"])


Counter({tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         tensor(True): 1,
         ten

In [10]:
from datasets import Dataset

# Convert train split to pandas
train_df = small_train.to_pandas()

# Check class counts
print("Before balancing:")
print(train_df["is_correct"].value_counts())

# Find smallest class count
min_count = train_df["is_correct"].value_counts().min()

# Sample equal counts from both classes
balanced_train_df = pd.concat([
    train_df[train_df["is_correct"] == True].sample(min_count, random_state=42),
    train_df[train_df["is_correct"] == False].sample(min_count, random_state=42)
])

# Convert back to HF Dataset
balanced_train = Dataset.from_pandas(balanced_train_df)

print("\nAfter balancing:")
print(balanced_train_df["is_correct"].value_counts())


Before balancing:
is_correct
False    7267
True     4733
Name: count, dtype: int64

After balancing:
is_correct
True     4733
False    4733
Name: count, dtype: int64


In [11]:
# Re-tokenize the balanced dataset
def preprocess_function(batch):
    texts = [
        "Question: " + q + " Answer: " + a
        for q, a in zip(batch["question"], batch["answer"])
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

tokenized_balanced_train = balanced_train.map(preprocess_function, batched=True, batch_size=256)

# Encode labels
def encode_labels(example):
    example["is_correct"] = 1 if example["is_correct"] else 0
    return example

tokenized_balanced_train = tokenized_balanced_train.map(encode_labels)
tokenized_balanced_train = tokenized_balanced_train.rename_column("is_correct", "labels")
tokenized_balanced_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Reuse training args from before
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_balanced_train
)

# Start fine-tuning again
trainer.train()


Map:   0%|          | 0/9466 [00:00<?, ? examples/s]

Map:   0%|          | 0/9466 [00:00<?, ? examples/s]

Step,Training Loss
500,0.493300
1000,0.473700


TrainOutput(global_step=1184, training_loss=0.48461667267051906, metrics={'train_runtime': 305.4519, 'train_samples_per_second': 30.99, 'train_steps_per_second': 3.876, 'total_flos': 626990530934784.0, 'train_loss': 0.48461667267051906, 'epoch': 1.0})

In [12]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Tokenize test set
def preprocess_function(batch):
    texts = [
        "Question: " + q + " Answer: " + a
        for q, a in zip(batch["question"], batch["answer"])
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

tokenized_test = small_test.map(preprocess_function, batched=True, batch_size=256)
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Get predictions
predictions = trainer.predict(tokenized_test)
pred_labels = np.argmax(predictions.predictions, axis=1)

# True labels (convert to int if needed)
true_labels = [1 if x else 0 for x in small_test["is_correct"]]

# Compute metrics
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print(classification_report(true_labels, pred_labels))


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Accuracy: 0.5013
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.50      0.67     10000

    accuracy                           0.50     10000
   macro avg       0.50      0.25      0.33     10000
weighted avg       1.00      0.50      0.67     10000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
submission_df = pd.DataFrame({
    "ID": range(len(pred_labels)),
    "is_correct": pred_labels
})

# Save again with the proper name
submission_df.to_csv("submission4.csv", index=False)

submission_df.head()


,ID,is_correct
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0
